# Clasificador de Carnes

## Maestría en Ciberseguridad

## Módulo: Tratamiento de Datos

### Desarrellado por: Iván Reyes

#### Importar Librerías

In [1]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator



#### Preparación de ciertos parámetros

In [2]:
#Tamaño de la Imagen
IMG_SIZE = 50
#Epocas
epochs = 15 
#muestras procesadas
batch_size = 64
#Archivo de almacenamineto del resumen del modelo
MODEL_SUMMARY_FILE = "model_summary.txt"
#Se estandariza el tamaño de las imagenes
IMAGE_WIDTH, IMAGE_HEIGHT = IMG_SIZE, IMG_SIZE

#### Preparación  Modelo Secuencial


In [4]:


model = Sequential([
    tf.keras.layers.Rescaling(1./255,input_shape = (IMG_SIZE,IMG_SIZE,3)),
    Conv2D(16,3,activation='relu',padding='same'),
    MaxPooling2D(),
    Conv2D(32,3,activation='relu',padding='same'),
    MaxPooling2D(),
    Conv2D(64,3,activation='relu',padding='same'),
    MaxPooling2D(),
    Flatten(),
    Dropout(0.5),
    Dense(128,activation='relu'),
    Dense(8)
])

optimizer = keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, 
                        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                        metrics=["accuracy"])

#### Almacenar el modelo en archivo

In [5]:

with open(MODEL_SUMMARY_FILE, "w") as fh:
    model.summary(print_fn=lambda line: fh.write(line + "\n"))

###### Resumen del modelo

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 50, 50, 3)         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 50, 50, 16)        448       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 25, 25, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 25, 25, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 12, 12, 64)        1

#### Preparación del DataSet 

#### Data Test

#### Data Validation

#### Data Train


In [10]:


train_ds = tf.keras.utils.image_dataset_from_directory('./train',seed =123,image_size=(IMAGE_WIDTH,IMAGE_HEIGHT))
classes_train = train_ds.class_names
num_classes = len(classes_train)

data_train_generator = ImageDataGenerator()
data_validation_generator = ImageDataGenerator()
data_test_generator = ImageDataGenerator()

test_generator = data_test_generator.flow_from_directory(
    './test',
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    class_mode='categorical',
    shuffle=False)

train_generator = data_train_generator.flow_from_directory(
    './train',
    target_size = (IMG_SIZE,IMG_SIZE),
    class_mode = 'categorical',
    shuffle=False)

validation_generator = data_validation_generator.flow_from_directory(
    './validation',
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    class_mode='categorical',
    shuffle=False)


Found 1633 files belonging to 8 classes.
Found 810 images belonging to 8 classes.
Found 1633 images belonging to 8 classes.
Found 996 images belonging to 8 classes.


#### Se entrena el modelo

In [11]:
classifier_meat = model.fit(train_ds,batch_size=batch_size,epochs=epochs)

Epoch 1/15
52/52 [==============================] - 4s 51ms/step - loss: 1.3211 - accuracy: 0.5769
Epoch 2/15
52/52 [==============================] - 2s 39ms/step - loss: 1.0076 - accuracy: 0.6216
Epoch 3/15
52/52 [==============================] - 2s 39ms/step - loss: 0.6101 - accuracy: 0.7581
Epoch 4/15
52/52 [==============================] - 2s 40ms/step - loss: 0.4856 - accuracy: 0.8108
Epoch 5/15
52/52 [==============================] - 2s 40ms/step - loss: 0.3864 - accuracy: 0.8518
Epoch 6/15
52/52 [==============================] - 2s 40ms/step - loss: 0.3814 - accuracy: 0.8512
Epoch 7/15
52/52 [==============================] - 2s 41ms/step - loss: 0.3936 - accuracy: 0.8420
Epoch 8/15
52/52 [==============================] - 2s 43ms/step - loss: 0.3233 - accuracy: 0.8836
Epoch 9/15
52/52 [==============================] - 2s 44ms/step - loss: 0.2705 - accuracy: 0.9045
Epoch 10/15
52/52 [==============================] - 3s 52ms/step - loss: 0.2199 - accuracy: 0.9277
Epoch 11/

#### Evaluación del Modelo y generación de matrices de confusión

In [12]:
#Predicción en test

predicted_test = model.predict(test_generator)

y_predict_test = np.argmax(predicted_test,axis=1)
y_real_test = test_generator.classes
correct_test = np.where(y_predict_test==y_real_test)[0]

print ('Valores de test correctos %s de %s'%(len(correct_test),len(y_real_test)))
print ('Porcentaje valores correctos: %.2f'%(len(correct_test)/len(y_real_test)*100))
confusion_matrix(y_real_test, y_predict_test)

26/26 [==============================] - 5s 180ms/step
Valores de test correctos 724 de 810
Porcentaje valores correctos: 89.38


array([[  0,   0,   0,   0,   0,   0,   1,   0],
       [  0,  43,   0,   5,   0,   0,   0,   0],
       [  0,   0,  54,   0,  11,   0,  32,   0],
       [  0,   2,   0,  42,   0,   0,   1,   0],
       [  0,   1,   2,   0, 448,   0,   5,   3],
       [  0,   1,   0,   0,   3,  15,   0,   0],
       [  0,   0,   1,   0,   3,   0, 110,   0],
       [  0,   0,   1,   0,  14,   0,   0,  12]], dtype=int64)

In [13]:
#Predicción en train

predicted_train = model.predict(train_generator)
y_predict_train = np.argmax(predicted_train,axis=1)
y_real_train = train_generator.classes
correct_train = np.where(y_predict_train==y_real_train)[0]

print ('Valores de train correctos  %s de %s'%(len(correct_train),len(y_real_train)))
print ('Porcentaje valores correctos: %.2f'%(len(correct_train)/len(y_real_train)*100))
confusion_matrix(y_real_train, y_predict_train)

52/52 [==============================] - 6s 108ms/step
Valores de train correctos  1530 de 1633
Porcentaje valores correctos: 93.69


array([[ 59,   0,   1,   0,   0,   2,   0],
       [  0, 157,   0,   9,   0,  47,   0],
       [  0,   0, 105,   0,   0,   0,   0],
       [  0,   2,   0, 937,   0,  10,   0],
       [  0,   0,   0,   1,  36,   0,   0],
       [  2,   0,   0,   0,   0, 202,   0],
       [  0,   0,   0,  29,   0,   0,  34]], dtype=int64)

#### Evaluación Final de Efectividad del Modelo

In [15]:
test_ds = tf.keras.utils.image_dataset_from_directory('./test', seed = 123, image_size = (IMAGE_WIDTH,IMAGE_HEIGHT))

train_eval = model.evaluate(train_ds, verbose=1)
test_eval = model.evaluate(test_ds, verbose=1)
print("\n")
print(f"Accuracy test: {round(100 * test_eval[1], 2)}% \n")
print(f"Accuracy train: {round(100 * train_eval[1], 2)}% \n")

Found 810 files belonging to 8 classes.
26/26 [==============================] - 1s 18ms/step - loss: 0.3605 - accuracy: 0.8864


Accuracy test: 88.64% 

Accuracy train: 95.22% 

